# Question 1 - API Response EDA

In [1]:
from collections import Counter
import country_converter as coco
import nltk
from nltk.corpus import stopwords 
import numpy as np
import fasttext
import pandas as pd
from project_utils.io import load_json_file
from scipy.spatial import distance
from sklearn.neighbors import DistanceMetric, NearestNeighbors

In [2]:
%load_ext autoreload
%autoreload
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bcf4k\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

---
## Examining Country Reporting

Let's begin with examining the country response (since it should be the simplest data).

In [3]:
all_occur_countries_d = load_json_file('data/all_occur_countries.json')
all_occur_countries_df = pd.DataFrame.from_dict(all_occur_countries_d)

For a count query of countries, there should only be one row for a given country.

In [4]:
all_occur_countries_df['term'].value_counts()

TL    1
ML    1
AN    1
MT    1
TH    1
     ..
BF    1
UG    1
VC    1
CI    1
FI    1
Name: term, Length: 235, dtype: int64

And, the data shows that.  Now, let's map the country codes to country name and retrieve the ISO3 country code for a future analysis.

In [5]:
iso2_countries = list(all_occur_countries_df['term'].values)
cc = coco.CountryConverter()

In [6]:
iso3_countries = pd.Series(cc.convert(iso2_countries, to='ISO3')).replace('not found', np.NaN)
countries_short_name = pd.Series(cc.convert(iso2_countries, to='short_name')).replace('not found', np.NaN)

In [7]:
all_occur_countries_df['iso3'] = iso3_countries
all_occur_countries_df['name'] = countries_short_name
all_occur_countries_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235 entries, 0 to 234
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   term    235 non-null    object
 1   count   235 non-null    int64 
 2   iso3    230 non-null    object
 3   name    230 non-null    object
dtypes: int64(1), object(3)
memory usage: 7.5+ KB


In [8]:
all_occur_countries_df

,term,count,iso3,name
0,US,6136862,USA,United States
1,GB,298758,GBR,United Kingdom
2,CA,260177,CAN,Canada
3,JP,259359,JPN,Japan
4,FR,250505,FRA,France
...,...,...,...,...
230,SX,1,SXM,Sint Maarten
231,TV,1,TUV,Tuvalu
232,VC,1,VCT,St. Vincent and the Grenadines
233,WF,1,WLF,Wallis and Futuna Islands


By being able to see the country names, we notice that the data does not fully aggregate countries in terms of sovereignty.  Wallis and Futuna Islands (as well as the Falklands) are counted separately from France and the UK (respectively).  This is not a major concern given the small number of records associated with these entities, but it is good to know.

The FDA resources indicate that adverse reaction counts should be suspect due to the underlying processes involved in their collection.  As a consequence of one of those processes, we would also expect to see a likely positive correlation between counts and population sizes. Let's examine that using 2018 data from the World Bank: https://data.worldbank.org/indicator/SP.POP.TOTL

In [9]:
pop_df = pd.read_csv('data/world_bank_pops.csv')
pop_df

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54211.0,55438.0,56225.0,56695.0,57032.0,57360.0,...,101669.0,102046.0,102560.0,103159.0,103774.0,104341.0,104872.0,105366.0,105845.0,NaN
1,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996973.0,9169410.0,9351441.0,9543205.0,9744781.0,9956320.0,...,29185507.0,30117413.0,31161376.0,32269589.0,33370794.0,34413603.0,35383128.0,36296400.0,37172386.0,NaN
2,Angola,AGO,"Population, total",SP.POP.TOTL,5454933.0,5531472.0,5608539.0,5679458.0,5735044.0,5770570.0,...,23356246.0,24220661.0,25107931.0,26015780.0,26941779.0,27884381.0,28842484.0,29816748.0,30809762.0,NaN
3,Albania,ALB,"Population, total",SP.POP.TOTL,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,...,2913021.0,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0,2873457.0,2866376.0,NaN
4,Andorra,AND,"Population, total",SP.POP.TOTL,13411.0,14375.0,15370.0,16412.0,17469.0,18549.0,...,84449.0,83747.0,82427.0,80774.0,79213.0,78011.0,77297.0,77001.0,77006.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Kosovo,XKX,"Population, total",SP.POP.TOTL,947000.0,966000.0,994000.0,1022000.0,1050000.0,1078000.0,...,1775680.0,1791000.0,1805200.0,1824100.0,1821800.0,1801800.0,1816200.0,1830700.0,1845300.0,NaN
260,"Yemen, Rep.",YEM,"Population, total",SP.POP.TOTL,5315355.0,5393036.0,5473671.0,5556766.0,5641597.0,5727751.0,...,23154855.0,23807588.0,24473178.0,25147109.0,25823485.0,26497889.0,27168210.0,27834821.0,28498687.0,NaN
261,South Africa,ZAF,"Population, total",SP.POP.TOTL,17099840.0,17524533.0,17965725.0,18423161.0,18896307.0,19384841.0,...,51216964.0,52004172.0,52834005.0,53689236.0,54545991.0,55386367.0,56203654.0,57000451.0,57779622.0,NaN
262,Zambia,ZMB,"Population, total",SP.POP.TOTL,3070776.0,3164329.0,3260650.0,3360104.0,3463213.0,3570464.0,...,13605984.0,14023193.0,14465121.0,14926504.0,15399753.0,15879361.0,16363507.0,16853688.0,17351822.0,NaN


In [10]:
all_countries_pop_df = pd.merge(all_occur_countries_df, 
                                pop_df[['Country Code', '2018']], 
                                left_on='iso3', 
                                right_on='Country Code')
all_countries_pop_df

,term,count,iso3,name,Country Code,2018
0,US,6136862,USA,United States,USA,326687501.0
1,GB,298758,GBR,United Kingdom,GBR,66460344.0
2,CA,260177,CAN,Canada,CAN,37057765.0
3,JP,259359,JPN,Japan,JPN,126529100.0
4,FR,250505,FRA,France,FRA,66977107.0
...,...,...,...,...,...,...
204,SO,1,SOM,Somalia,SOM,15008154.0
205,SX,1,SXM,Sint Maarten,SXM,40654.0
206,TV,1,TUV,Tuvalu,TUV,11508.0
207,VC,1,VCT,St. Vincent and the Grenadines,VCT,110210.0


In [11]:
set(all_occur_countries_df['term'].values) - set(all_countries_pop_df['term'].values)

{'AI',
 'AN',
 'AQ',
 'AX',
 'BL',
 'BQ',
 'CX',
 'FK',
 'FX',
 'GF',
 'GG',
 'GP',
 'IO',
 'JE',
 'MQ',
 'PM',
 'PN',
 'QZ',
 'RE',
 'SH',
 'TP',
 'TW',
 'UM',
 'WF',
 'YT',
 'YU'}

We lost some countries in the join, but this is due to the more granular political reporting of the FDA data and, given this is just a data exploration, it isn't a concern.

In [12]:
all_countries_pop_df[['count', '2018']].corr()

,count,2018
count,1.000000,0.160503
2018,0.160503,1.000000


Surprisingly, we see a very weak correlation, but this could reflect cultural and economic characteristics (the more dissimilar to the US, the less reporting).

In [13]:
all_countries_pop_df[~all_countries_pop_df['name'].isin(['China', 'India'])][['count', '2018']].corr()

,count,2018
count,1.000000,0.493739
2018,0.493739,1.000000


Sure enough, we see outlier effects.

Let's remove a few more large and "different" countries.

In [14]:
all_countries_pop_df[~all_countries_pop_df['name'].isin(['China', 'India', 'Bangladesh', 'Nigeria'])][['count', '2018']].corr()

,count,2018
count,1.00000,0.52914
2018,0.52914,1.00000


And it goes up some more.

Now, let's trying using a more robust correlation method (without dropping any countries).

In [15]:
all_countries_pop_df[['count', '2018']].corr(method='spearman')

,count,2018
count,1.000000,0.682119
2018,0.682119,1.000000


So what is the significance of this?  It just highlights an additional factor that impacts adverse event reporting across countries. **For this reason, an implementation will avoid relying on comparisons that rely on FAERS counts.**

---
## Examining Adverse Event Labeling

For patient reactions, we do expect multiple rows with the same reaction. However, because these are (presumably) just human-entered form inputs, it would be good to have an understanding of how well the text data was standardized.

In [16]:
all_patient_reactions_d = load_json_file('data/all_patient_reactions.json')
all_patient_reactions_df = pd.DataFrame.from_dict(all_patient_reactions_d)
all_patient_reactions_df['term'] = all_patient_reactions_df['term'].str.lower()

In [17]:
all_patient_reactions_df['term'].sort_values()[:50]

289                                 abasia
64                    abdominal discomfort
98                    abdominal distension
29                          abdominal pain
408                   abdominal pain lower
42                    abdominal pain upper
225                     abnormal behaviour
325                        abnormal dreams
565                   abnormal weight gain
777                       abortion induced
259                   abortion spontaneous
649                                abscess
830                               accident
235         accidental exposure to product
329                    accidental overdose
152                                   acne
353    activities of daily living impaired
880                acute coronary syndrome
734                  acute hepatic failure
77                     acute kidney injury
641                acute myeloid leukaemia
326            acute myocardial infarction
582    acute respiratory distress syndrome
605        

Looking at the results, we can see that indices 408 ("ABDOMINAL PAIN LOWER") and 42 ("ABDOMINAL PAIN UPPER") are both types of 29 ("ABDOMINAL PAIN").  Thus, we know that the OpenFDA API does not have this kind of standardization.

---
### Deduplication

Now, let's look at the bottom of the series.

In [18]:
all_patient_reactions_df['term'].sort_values(ascending=False)[:50]

68     wrong technique in product usage process
190       wrong technique in drug usage process
561     wrong technique in device usage process
668                     wrong drug administered
773                              wrist fracture
968                             wound infection
556                                       wound
313                         withdrawal syndrome
276            white blood cell count increased
106            white blood cell count decreased
249                                    wheezing
32                             weight increased
24                             weight decreased
11                                     vomiting
997                           vitreous floaters
935                        vitamin d deficiency
959                         vitamin d decreased
94                            visual impairment
980                         visual field defect
878                          visual disturbance
256                       visual acuity 

Row-pairs like 276-106 and 32-24 are very similar, but the last token makes these opposites (semantically speaking), so we would lose meaning by simply truncating the levels. However, if we calculate metrics on both the first and last tokens, then this may provide us with two conservative estimates of how many unique reaction events exist.

In [19]:
split_reactions_series = all_patient_reactions_df['term'].str.split()
first_tokens = split_reactions_series.apply(lambda x: x[0])
last_tokens = split_reactions_series.apply(lambda x: x[-1])

In [20]:
first_tokens[:15]

0           drug
1         nausea
2          death
3        fatigue
4       headache
5      diarrhoea
6       dyspnoea
7           pain
8            off
9      dizziness
10       malaise
11      vomiting
12          rash
13      asthenia
14    arthralgia
Name: term, dtype: object

In [21]:
last_tokens[:15]

0     ineffective
1          nausea
2           death
3         fatigue
4        headache
5       diarrhoea
6        dyspnoea
7            pain
8             use
9       dizziness
10        malaise
11       vomiting
12           rash
13       asthenia
14     arthralgia
Name: term, dtype: object

In [22]:
first_tokens_set = set(first_tokens)
last_tokens_set = set(last_tokens)
num_common = len(first_tokens_set & last_tokens_set)
num_common

341

In [23]:
num_common/len(first_tokens_set)

0.5387045813586098

In [24]:
num_common/len(last_tokens_set)

0.6144144144144145

We see that these will have limited efficacy, as many labels (from this example) are only a single token. Regardless, it still provides a naive summary baseline.

Another step we can take is to remove English stop words. While we are at it, we will also remove any unordered, duplicate tokens.

In [25]:
en_stop_words = stopwords.words('english')
combined_hash_d = {}    # Frozenset dict to identify equal unordered patient reactions
dropped_patient_reactions = set()    # Duplicated patient reactions to remove
no_stop_words_patient_reactions = []    # Patient reactions without stop words

for reaction in all_patient_reactions_df['term'].values:
    no_stop_word_tokens = [token for token in reaction.split() if token not in en_stop_words]
    tokenized_event = frozenset(no_stop_word_tokens)
    if tokenized_event not in combined_hash_d:
        combined_hash_d[tokenized_event] = 1
        no_stop_words_patient_reactions.append(' '.join(no_stop_word_tokens))
    else:
        dropped_patient_reactions = dropped_patient_reactions | set([reaction])

Unforunately, it did not make much of a difference here. However, it may be useful when comparing the countries directly.

In [26]:
dropped_patient_reactions

{'adverse event'}

In [27]:
deduped_reactions_df = all_patient_reactions_df[~all_patient_reactions_df['term'].isin(dropped_patient_reactions)]
deduped_reactions_df.loc[:, 'term'] = no_stop_words_patient_reactions
deduped_reactions_df.shape

c:\users\bcf4k\.virtualenvs\astrazeneca-xwrglwwk\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


(999, 2)

### Additional token metrics

Since we have already tokenized the labels, let's go ahead and take a look at related metrics.

In [28]:
tokens = set()
token_lens = []
all_tokens = []
for key in combined_hash_d.keys():
    cur_tokens = [token for token in key]
    token_lens.append(len(cur_tokens))
    all_tokens += cur_tokens
    tokens = tokens | set(cur_tokens)
token_lens = np.array(token_lens)

In [29]:
tokens_counter = Counter(all_tokens)
tokens_counter.most_common(35)

[('disorder', 40),
 ('increased', 38),
 ('decreased', 35),
 ('blood', 34),
 ('drug', 32),
 ('pain', 31),
 ('site', 26),
 ('product', 26),
 ('infection', 21),
 ('device', 20),
 ('injection', 17),
 ('abnormal', 17),
 ('haemorrhage', 17),
 ('skin', 16),
 ('syndrome', 16),
 ('failure', 14),
 ('disease', 14),
 ('injury', 13),
 ('respiratory', 12),
 ('fracture', 12),
 ('error', 11),
 ('rash', 10),
 ('discomfort', 10),
 ('renal', 10),
 ('pulmonary', 10),
 ('count', 10),
 ('issue', 9),
 ('administered', 9),
 ('acute', 9),
 ('reaction', 9),
 ('eye', 9),
 ('muscle', 8),
 ('oedema', 8),
 ('swelling', 8),
 ('cell', 7)]

Some of these seem like fairly generic medical words and with a deeper examination could become domain-specific stopwords.

In [30]:
pd.Series(token_lens).describe()

count    999.000000
mean       2.036036
std        0.894597
min        1.000000
25%        1.000000
50%        2.000000
75%        3.000000
max        6.000000
dtype: float64

This just confirms that the labels are just a few tokens.

In [31]:
len(tokens)

963

And we have fewer unique tokens than there are events, so it is a limited vocabulary (but very domain-specific).

### fastText Embedding Approach

Let's see if there is anything we can do with these events using a domain-specific fastText embedding (https://github.com/ncbi-nlp/BioSentVec#biowordvec).

In [32]:
bioword_m = fasttext.load_model('models/BioWordVec_PubMed_MIMICIII_d200.bin')

Let's see how well the embeddings model works first.  

We'll begin by examining performance on individual tokens.

In [33]:
token_vectors = np.array([bioword_m[token] for token in tokens])
token_vectors.shape

(963, 200)

To use the `sklearn.neighbors.NearestNeighbors` implementation, we need to define a cosine metric.

In [34]:
def cosine_similarity(x, y):
    return distance.cosine(x, y)
cosine_dist = DistanceMetric.get_metric(cosine_similarity)

In [35]:
nn = NearestNeighbors(metric=cosine_similarity)
nn.fit(token_vectors)

NearestNeighbors(metric=<function cosine_similarity at 0x000001260CA67B88>)

Notice we are only using a data split because we are not trying to perform an inference operation.

Let's make a few helper functions to help view the results.

In [36]:
tokens_lookup = list(tokens)

def get_token(index, lookup):
    return lookup[index]

def get_vector(index, vectors):
    return vectors[index].reshape(1,-1)

def query_nn(q_token, lookup, vectors, n=10):
    dists, indices = nn.kneighbors(get_vector(lookup.index(q_token), vectors), 
                                   n_neighbors=n)
    nn_tokens = []
    for index in indices[0]:
        nn_tokens.append(get_token(index, lookup))
    nn_df = pd.DataFrame()
    nn_df['tokens'] = nn_tokens
    nn_df['dists'] = dists[0]
    return nn_df

In [37]:
query_nn('cardiovascular', tokens_lookup, token_vectors)

,tokens,dists
0,cardiovascular,0.000000
1,cerebrovascular,0.267182
2,cardiac,0.328129
3,heart,0.332133
4,cardio-respiratory,0.333668
5,hypertension,0.353343
6,coronary,0.374313
7,obesity,0.388123
8,hypertensive,0.397763
9,arteriosclerosis,0.412435


In [38]:
query_nn('renal', tokens_lookup, token_vectors)

,tokens,dists
0,renal,0.000000
1,kidney,0.168924
2,nephropathy,0.309620
3,tubulointerstitial,0.313288
4,glomerular,0.313915
5,nephrolithiasis,0.318619
6,anuria,0.327387
7,tubular,0.332714
8,proteinuria,0.342649
9,pyelonephritis,0.364901


So far so good.

In [39]:
query_nn('disease', tokens_lookup, token_vectors)

,tokens,dists
0,disease,0.000000
1,crohn's,0.406441
2,progression,0.412011
3,sclerosis,0.426895
4,ulcerative,0.428422
5,illness,0.428868
6,vasculitis,0.437415
7,arteriosclerosis,0.438886
8,arthropathy,0.454075
9,progressive,0.460800


In [40]:
query_nn('increased', tokens_lookup, token_vectors)

,tokens,dists
0,increased,0.000000
1,decreased,0.050103
2,reduced,0.134189
3,lower,0.300187
4,altered,0.302783
5,improved,0.328971
6,impaired,0.376512
7,associated,0.391276
8,aggravated,0.395201
9,change,0.406824


In [41]:
query_nn('abdominal', tokens_lookup, token_vectors)

,tokens,dists
0,abdominal,0.000000
1,chest,0.313670
2,pelvic,0.331445
3,distension,0.356398
4,hernia,0.357802
5,ileus,0.384067
6,appendicitis,0.394053
7,bowel,0.401358
8,neck,0.402837
9,groin,0.413477


These latter three embeddings are not bad, but they do demonstrate some potentially troubling characteristics. For example, it's not ideal (euphemestically) for "increased" and "decreased" to be _so_ close.

Okay, well now let's take a look at an embedding for a whole phrase.  Because the model is meant for tokens, we don't want to embed the whole event label. Instead, we will get the average embedding for the event.

In [42]:
def calc_avg_embed(label):
    tokens = label.split()
    num_tokens = len(tokens)
    embed = np.zeros((1,200))
    for token in tokens:
        embed += bioword_m[token].reshape(1,-1)
    return embed/num_tokens

In [43]:
avg_embeds = []
for reaction in deduped_reactions_df['term'].values:
    avg_embeds.append(calc_avg_embed(reaction))
avg_embeds = np.array(avg_embeds).reshape(deduped_reactions_df.shape[0], -1)

In [44]:
nn = NearestNeighbors(metric=cosine_similarity)
nn.fit(avg_embeds)

NearestNeighbors(metric=<function cosine_similarity at 0x000001260CA67B88>)

In [45]:
query_nn('acute kidney injury', no_stop_words_patient_reactions, avg_embeds)

,tokens,dists
0,acute kidney injury,0.000000
1,renal injury,0.080062
2,renal failure acute,0.134533
3,liver injury,0.145278
4,drug-induced liver injury,0.162303
5,injury,0.175621
6,chronic kidney disease,0.178010
7,renal failure chronic,0.178440
8,acute hepatic failure,0.179313
9,brain injury,0.192022


In [46]:
query_nn('alanine aminotransferase increased', no_stop_words_patient_reactions, avg_embeds)

,tokens,dists
0,alanine aminotransferase increased,0.000000
1,aspartate aminotransferase increased,0.023540
2,transaminases increased,0.120841
3,gamma-glutamyltransferase increased,0.155268
4,blood lactate dehydrogenase increased,0.248330
5,blood creatine phosphokinase increased,0.248375
6,hepatic enzyme increased,0.265477
7,blood alkaline phosphatase increased,0.278147
8,lipase increased,0.302263
9,blood creatinine increased,0.308782


In [47]:
query_nn('vaginal haemorrhage', no_stop_words_patient_reactions, avg_embeds)

,tokens,dists
0,vaginal haemorrhage,0.000000
1,genital haemorrhage,0.101093
2,rectal haemorrhage,0.138552
3,uterine perforation,0.192724
4,haemorrhage,0.200422
5,gastrointestinal haemorrhage,0.209894
6,gastric haemorrhage,0.221301
7,vaginal discharge,0.222550
8,skin haemorrhage,0.229548
9,haemorrhage intracranial,0.246088


This looks pretty good as well.

With this information in mind, let's move to a solution.